In [1]:
import numpy as np
import time

# --- Constantes do Projeto (Baseado no VHDL) ---
C_IN_WIDTH = 16
C_ACC_WIDTH = 24
# O pior caso de produto: 127 * 127 = 16129
C_MAX_PROD = np.int16(16129)
# O pior caso de produto negativo: -128 * 127 = -16256
C_MIN_PROD = np.int16(-16256) 

class Accumulator24Bit:
    """
    Simula o Acumulador síncrono de 24 bits com entrada de 16 bits.
    Utiliza np.int32 para simular o armazenamento de 24 bits com sinal.
    """
    
    def __init__(self):
        # O valor é inicializado em 0. np.int32 é usado para representar 24 bits com sinal.
        self.value = np.int32(0) 

    def update(self, P_in: np.int16, acc_reset: int, acc_enable: int) -> np.int32:
        """ 
        Simula a lógica síncrona do acumulador (rising_edge(clk)).
        
        P_in: Produto de 16 bits.
        acc_reset: Sinal de reset (1=reset).
        acc_enable: Sinal de enable (1=acumula).
        
        Retorna o valor atual do acumulador (24 bits).
        """
        
        # 1. Lógica de Reset (Alta Prioridade)
        if acc_reset == 1:
            self.value = np.int32(0)
            return self.value

        # 2. Lógica de Acumulação (Apenas se Enabled)
        if acc_enable == 1:
            # A extensão de sinal (Sign Extension) de 16 bits para 24 bits é feita 
            # implicitamente na soma, mas garantimos que P_in é tratado como 16 bits.
            
            # P_in é explicitamente somado ao valor atual. O resultado, 
            # por ser armazenado em np.int32, simula o espaço de 24 bits (e verifica o overflow).
            
            # Nota: Em VHDL, a soma 24-bit + 16-bit (estendido) é o que ocorre.
            self.value += np.int32(P_in)
            
            # 3. Simulação do clipping de 24 bits (Embora o projeto diga que não deve haver overflow, 
            # é uma verificação de segurança, mas aqui confiamos no np.int32)
            # Para 24 bits, o máximo é 2^23 - 1 e o mínimo é -2^23
            # MAX_24 = 8388607 (0x7FFFFF), MIN_24 = -8388608 (0x800000)
            
        # O valor de retorno é o valor atual do registrador.
        return self.value

# --- Função de Execução de Teste ---

def run_test_cycle(acc_instance, P_in, acc_reset, acc_enable, expected_total, test_desc, test_id):
    """Executa um ciclo de clock na simulação."""
    
    start_time = time.perf_counter()
    # Executa a lógica síncrona (próximo estado)
    resultado = acc_instance.update(P_in, acc_reset, acc_enable)
    end_time = time.perf_counter()
    duration = (end_time - start_time) * 1e6 # µs

    # Trunca o resultado para 24 bits (para comparação)
    # 0xFFFFFF é 24 bits. Aplicamos a conversão para np.int32 novamente
    # para garantir a interpretação de sinal de 24 bits (se o bit 23 estiver setado)
    result_24bit_masked = np.int32((resultado & 0xFFFFFF) | (resultado & 0x800000) * 0x1FE) # Not strictly needed if np.int32 works.
    
    verification = '✅ PASSOU' if resultado == expected_total else '❌ FALHOU'
    
    print(f"\n--- Teste {test_id}: {test_desc} ---")
    print(f"  P_in (16b): {P_in} | Reset: {acc_reset} | Enable: {acc_enable}")
    print(f"  Resultado Saída (24b): {resultado} | Hex: {hex(np.uint32(resultado))}")
    print(f"  Valor Esperado (24b): {expected_total} | Hex: {hex(np.uint32(expected_total))}")
    print(f"  Tempo de Execução (Python): {duration:.3f} µs")
    print(f"  Verificação: {verification}")
    
    return resultado

# --- Execução dos Casos de Teste (Simulação Completa) ---

print("## Testbench Acumulador 24 bits - Testes Unificados ##")
acc = Accumulator24Bit()
current_total = np.int32(0) # Variável de controle do valor esperado

# 1. Teste de Reset (acc_reset='1')
current_total = np.int32(0)
run_test_cycle(acc, P_in=np.int16(1000), acc_reset=1, acc_enable=0, expected_total=current_total,
               test_desc="1: RESET (Deveria ser 0)", test_id=1)

# 2. Acumulação (acc_enable='1') - Mapeando os testes VHDL (2, 3, 4, 5)

# Teste 2: ACUMULACAO 1/4 (EXEC_DOT_0)
P_IN_2 = np.int16(1000)
current_total += P_IN_2
run_test_cycle(acc, P_in=P_IN_2, acc_reset=0, acc_enable=1, expected_total=current_total,
               test_desc="2: ACUMULACAO 1/4 (Soma 1000)", test_id=2)

# Teste 3: ACUMULACAO 2/4 (EXEC_DOT_1)
P_IN_3 = np.int16(5000)
current_total += P_IN_3
run_test_cycle(acc, P_in=P_IN_3, acc_reset=0, acc_enable=1, expected_total=current_total,
               test_desc="3: ACUMULACAO 2/4 (Soma 5000)", test_id=3)

# Teste 4: ACUMULACAO 3/4 (Negativo) (EXEC_DOT_2)
P_IN_4 = np.int16(-2000)
current_total += P_IN_4
run_test_cycle(acc, P_in=P_IN_4, acc_reset=0, acc_enable=1, expected_total=current_total,
               test_desc="4: ACUMULACAO 3/4 (Soma -2000)", test_id=4)

# Teste 5: Acumulação 4/4 (Fim DOT) (EXEC_DOT_3)
P_IN_5 = np.int16(3000)
current_total += P_IN_5
run_test_cycle(acc, P_in=P_IN_5, acc_reset=0, acc_enable=1, expected_total=current_total,
               test_desc="5: ACUMULACAO 4/4 (Fim DOT)", test_id=5)

# 3. Teste de Desabilitação (acc_enable='0')
# Deve reter o valor anterior (1000 + 5000 - 2000 + 3000 = 7000)
P_IN_DISABLE = np.int16(9999) # Tenta somar um valor grande
run_test_cycle(acc, P_in=P_IN_DISABLE, acc_reset=0, acc_enable=0, expected_total=current_total,
               test_desc="6: DESABILITADO (Deve reter 7000)", test_id=6)


# 4. Pior Caso (Overflow Check - 4x Max Prod)
print("\n--- Testes de Borda e Limite Superior ---")

# Reset para novo cálculo (Simulando VHDL Teste 6)
acc.update(P_in=np.int16(0), acc_reset=1, acc_enable=0)
current_total = np.int32(0) 

# Somas do valor máximo (4x 16129 = 64516)
P_MAX = C_MAX_PROD

current_total += P_MAX
run_test_cycle(acc, P_in=P_MAX, acc_reset=0, acc_enable=1, expected_total=current_total,
               test_desc="7: LIMITE 1/4 (Soma Max Prod)", test_id=7)
current_total += P_MAX
run_test_cycle(acc, P_in=P_MAX, acc_reset=0, acc_enable=1, expected_total=current_total,
               test_desc="8: LIMITE 2/4", test_id=8)
current_total += P_MAX
run_test_cycle(acc, P_in=P_MAX, acc_reset=0, acc_enable=1, expected_total=current_total,
               test_desc="9: LIMITE 3/4", test_id=9)
current_total += P_MAX
run_test_cycle(acc, P_in=P_MAX, acc_reset=0, acc_enable=1, expected_total=current_total,
               test_desc="10: LIMITE 4/4 (Total 64516)", test_id=10)

# 5. Caso de Borda (Underflow Check - 4x Min Prod)
print("\n--- Testes de Borda e Limite Inferior ---")

# Reset
acc.update(P_in=np.int16(0), acc_reset=1, acc_enable=0)
current_total = np.int32(0) 

# Somas do valor mínimo negativo (4x -16256 = -65024)
P_MIN = C_MIN_PROD

current_total += P_MIN
run_test_cycle(acc, P_in=P_MIN, acc_reset=0, acc_enable=1, expected_total=current_total,
               test_desc="11: UNDERFLOW 1/4 (Soma Min Prod)", test_id=11)
current_total += P_MIN
run_test_cycle(acc, P_in=P_MIN, acc_reset=0, acc_enable=1, expected_total=current_total,
               test_desc="12: UNDERFLOW 2/4", test_id=12)
current_total += P_MIN
run_test_cycle(acc, P_in=P_MIN, acc_reset=0, acc_enable=1, expected_total=current_total,
               test_desc="13: UNDERFLOW 3/4", test_id=13)
current_total += P_MIN
run_test_cycle(acc, P_in=P_MIN, acc_reset=0, acc_enable=1, expected_total=current_total,
               test_desc="14: UNDERFLOW 4/4 (Total -65024)", test_id=14)

## Testbench Acumulador 24 bits - Testes Unificados ##

--- Teste 1: 1: RESET (Deveria ser 0) ---
  P_in (16b): 1000 | Reset: 1 | Enable: 0
  Resultado Saída (24b): 0 | Hex: 0x0
  Valor Esperado (24b): 0 | Hex: 0x0
  Tempo de Execução (Python): 1.300 µs
  Verificação: ✅ PASSOU

--- Teste 2: 2: ACUMULACAO 1/4 (Soma 1000) ---
  P_in (16b): 1000 | Reset: 0 | Enable: 1
  Resultado Saída (24b): 1000 | Hex: 0x3e8
  Valor Esperado (24b): 1000 | Hex: 0x3e8
  Tempo de Execução (Python): 3.700 µs
  Verificação: ✅ PASSOU

--- Teste 3: 3: ACUMULACAO 2/4 (Soma 5000) ---
  P_in (16b): 5000 | Reset: 0 | Enable: 1
  Resultado Saída (24b): 6000 | Hex: 0x1770
  Valor Esperado (24b): 6000 | Hex: 0x1770
  Tempo de Execução (Python): 2.400 µs
  Verificação: ✅ PASSOU

--- Teste 4: 4: ACUMULACAO 3/4 (Soma -2000) ---
  P_in (16b): -2000 | Reset: 0 | Enable: 1
  Resultado Saída (24b): 4000 | Hex: 0xfa0
  Valor Esperado (24b): 4000 | Hex: 0xfa0
  Tempo de Execução (Python): 1.500 µs
  Verificação: ✅ PASSOU

---

C:\Users\rebec\AppData\Local\Temp\ipykernel_10380\3401011493.py:71: RuntimeWarning: overflow encountered in scalar multiply
  result_24bit_masked = np.int32((resultado & 0xFFFFFF) | (resultado & 0x800000) * 0x1FE) # Not strictly needed if np.int32 works.


np.int32(-65024)